The first chapter itemize a time series into three components: `trend`, `seasonal` and `residual`. This chapter considers the later, a.k.a `cycle`, in all of the pivotal traits thoroughly developed. A beginner yet fundamental model leverages `White Noise` as building blocks. Let $y_t$ calculated as:

$$y_t = \epsilon_t $$ 
$$ \epsilon_t \sim N(0, \sigma ^2 ) $$

The "error" $\epsilon$ is uncorrelated over time. That process with zero mean, constant variance, and no correlation, is known as `zero-mean gaussian white noise`. Therefore: 

$$ y_t \sim WN(0, \sigma ^2 ) $$

In addition to it, autocovariances structures are smoothly calculated. Let $\tau$ as a displacement, the autocovariance function $\gamma$ is. 

\begin{equation}
\gamma(\tau)=
\begin{cases}
  \sigma ^2, & \tau = 0
\\
  0, & \tau \geq 1
\end{cases}
\end{equation}

The autocorrelation function for a white noise:


\begin{equation}
\rho(\tau)=
\begin{cases}
  \sigma ^2, & \tau = 0
\\
  0, & \tau \geq 1
\end{cases}
\end{equation}

Finally, the partial autocorrelation function is:


\begin{equation}
p(\tau)=
\begin{cases}
  \sigma ^2, & \tau = 0
\\
  0, & \tau \geq 1
\end{cases}
\end{equation}

We've itemized white noise in terms of its mean, variance, autocorrelation, and partial autocorrelation function. Notwithstanding the staggering magnitude of these traits, they don't craft an azimuth to takeaways from both the first and second moments regarding the process; the past `conditional` mean and variance do. We shall distinguish `unconditional` and `conditional` mean and variance.    


Let the conditioned mean and variance upon the information $\Omega_{t-1}$, which contains a past history of the observed series. It might leverage observed series, $\Omega_{t-1}$ = $y_{t-1}$, $y_{t-2}$,...; or the residuals(shocks), $\Omega_{t-1}$ = $\epsilon_{t-1}$, $\epsilon_{t-2}$,... 

Time to Estimation and Inference for the Mean, Autocorrelation and Partial Autocorrelation Functions.   

1) The estimator for the population mean, given a sample of size T.   


$$ \bar{y} = \dfrac{1}{Y}\sum_{t=1}^{T} y_t $$   


2) Sample Autocorrelations.   

$$ \hat{\rho}(\tau) = \dfrac{\sum_{t=\tau+1}^{T}[(y_t-\hat{y})(y_{t-\tau}-\hat{y}) ]}{\sum_{t=1}^{T}(y_t-\hat{y})^2}. $$


It is usual to assess whether a series is reasonably a white noise. The distribution of the sample autocorrelations in large samples is:    


$$ \hat{\rho}(\tau) \sim N(0, \dfrac{1}{T} ). $$


Under normality, the 95% confidence interval is build by two standard errors around the mean, making up the boundaries: $0 \pm 2/\sqrt{T}$.    

3) Sample Partial Autocorrelations.    


They are obtained from population linear regressions. If the fitted regression is:


$$ \hat{y_t} = \hat{c} + \hat{\beta_1}y_{t-1} + ... + \hat{\beta_\tau}y_{t-\tau}. $$


the `sample partial autocorrelation` at displacement $\tau$ is:

$$ \hat{p}(\tau) \equiv \hat{\beta_\tau}. $$

A straightforward solution for this is by least squares analytical solution.

$$ \beta=(X^TX)^{-1}X^Ty.$$ 


In [65]:
import numpy as np

def generate_white_noise(size: int):
    return np.random.normal(loc=0, scale=0.5, size=size)

def mean_estimator(time_series: np.array):
    return np.mean(time_series)

def sample_autocorrelation(time_series: np.array, tau: int):
    y_hat = mean_estimator(time_series)
    residuals = time_series - y_hat
    
    numerator = np.sum(residuals[tau+1:] * residuals[1:len(residuals)-tau])
    denominator = np.sum(residuals[1:]**2)      
      
    return numerator/denominator

from numpy.linalg import inv, solve

def sample_partial_autocorrelation(X_lags : np.array, y : np.array):
    return np.dot(inv(np.dot(X_lags.T, X_lags)), np.dot(X_lags.T, y))

In [18]:
sample_syntetic = generate_white_noise(40)

print('mean:', mean_estimator(sample_syntetic))
print('autocorrelation:', sample_autocorrelation(sample_syntetic, 0))

mean: -0.026428980620308646
autocorrelation: 1.0


In [61]:
from sktime.transformations.series.lag import Lag

## lags for a partial autocorrelation plot for a p=5 analysis.
t = Lag([1, 2, 3, 4, 5])
Xt = t.fit_transform(sample_syntetic)
sample_syntetic_lags = Xt[:-5]

In [64]:
sample_partial_autocorrelation(sample_syntetic_lags[5:], sample_syntetic[5:])

array([-0.19242129, -0.21358846,  0.01239841, -0.10850784,  0.11303483])

The `Lag Operator` lies in the gritty of time series models. It operates on a series by lagging it. It is usual to operate not with lag operator, but with its polynomial. A lag operator of degree m is just a linear function of powers of $L$, up through the $m-th$ power.

$$ B(L) = b_0 + b_1L + b_2L^2 + ... + b_mL^m.$$

Furthermore, infinite-order polynomials are super useful as well. Models involving infinite distributed lags are central to time series modeling.

### Autoregressive Processes


It is a natural model for cycles. In AR Model, the current value is linearly related to its past values and a stochastic residual/shock. The first-order autoregressive process, AR(1), is:

$$y_t = \phi y_{t-1} + \epsilon_t$$
$$\epsilon_t \sim WN(0, \sigma^2) $$

Even a simple AR(1) can be plastic and resilient. Let's take a peek at the shapes of two simulations with parameters $\sigma$ as `0.95` and `0.4`. 

In [ ]:
from statsmodels.tsa.arima_process import ArmaProcess

import matplotlib.pyplot as plt


plt.subplot(2, 1, 1)
ar1 = np.array([1, 0.9])
ma1 = np.array([1])
AR_object1 = ArmaProcess(ar1, ma1)
simulated_data_1 = AR_object1.generate_sample(nsample=150)
plt.plot(simulated_data_1);

# Plot 2: AR parameter = -0.9
plt.subplot(2, 1, 2)
ar2 = np.array([1, 0.4])
ma2 = np.array([1])
AR_object2 = ArmaProcess(ar2, ma2)
simulated_data_2 = AR_object2.generate_sample(nsample=150)
plt.plot(simulated_data_2);

: 

There is a natural expasion for the Autoregressive model. Assuming the AR(1) process, written as:

$$ y_t = \phi y_{t-1} + \epsilon_t$$

if we replace the lagged y's with backward, we end up with the following model.

$$ y_t = \phi y_{t-2} + \phi \epsilon_{t-1}  + \epsilon_t $$

$$...$$

$$ y_t = \epsilon_t + \phi \epsilon_{t-1} + \phi^2 \epsilon_{t-2} + ...$$

This is the `moving-average model` representation.  

That model is intuitive. As long as $\epsilon$'s are the components that move $y$, we would be able to express $y$ in terms of the history of $\epsilon$.

##### The AR(p) Process 

The general p-th order autoregressive process is:

$$y_t = \phi_1 y_{t-1} + \phi_2 y_{t-2} + ... + \phi_p y_{t-p} + \epsilon_t $$
$$ \epsilon_t \sim WN(0, \sigma^2) $$


Broadly speaking, the autocorrelation function AR(p) decays gradually throughout displacement. In addition to it, an AR(p) process is covariance stationary if and only if all roots of lag operator polynomial are outside the unit circle.

$$ \Phi (L)y_t = (1 - \phi_1 L - \phi_2 L^2 - ... - \phi_p L^p)y_t. $$ 


----- 
##### Forecasting Cycles with Autoregressions


The first key component is the `feature set`, which holds the available historical data. The T-time information set $\Omega_T$ is represented as:

$$ \Omega_T = {y_T, y_{T-1}, y_{T-2}, ...} $$

We aim to project the `optimal forecast` for a future time `T + h`, given the `information set`. The optimal is the that one minimizes the expected loss, or the conditional mean. 

$$ E(y_{T+H} | \Omega_T) $$

The conditional mean doesn't need to be a linear function; however, for AR sake, it is. As long as forecasts are linear projections in the elements on the information set, denoted as

$$ P(y_{T+h} | \Omega_T) $$ 

In the Gaussian case, the conditional expectation is exactly linear, so that.

$$ E(y_{T + h} | \Omega_T) = P(y_{T + h} | \Omega_T ) $$


-----------------------------
#### Point Forecasts for Autoregressions

Taking an AR(1) process, $y_t = \phi y_{t-1} + \epsilon_t$; $\epsilon \sim WN(0, \sigma^2)$.

To construct a 1-step-ahead forecast, the preocess for T + 1,

$$y_{T+1} = \phi y_{T} + \epsilon_{T+1}. $$

Transforming the right-hand side on the time-T information set.

$$y_{T+1,T} = \phi y_{T}. $$

Then, for the prediction of h displacement, 

$$y_{T+h,T} = \phi^h y_{T}. $$

As long as Normality is an assumption, we may draw a h-step-ahead forecast by means of a Gaussian realization with parameters:

$$ y_{T+h} \sim N(y_{T+h,T}, \sigma_h^2), $$

where $\sigma_h² = var(y_{T+h}|\Omega_{T})$ and $\Omega_T$ = {$y_T, y_{T-1}, ...$}. Using Wold's chain rule, the variance for a h-step-ahead forecast error variance, $\sigma_h^2$, for an AR(1) process.


$$ \sigma_1^2 = \sigma^2 $$
$$ \sigma_2^2 = \sigma^2(1 + \phi^2)$$
$$ \sigma_h^2 = \sigma^2 \sum_{i=0}^{h-1} \phi^{2i}. $$


By a known convergence, we it is possible to assess:

$$\lim_{x\to\infty} \sigma_h ^2 = \frac{\sigma^2}{1-\sigma^2}$$


-------

To knock off this section, we instill on a crucial concept regarding covariance stationary time-series, the `Wold Representation`. To recap and motivate the subject, a time-series can be decomposed into three main components: Trend, Seasonal and Residual, in which the later is generally `a zero-mean covariance-stationary process`. We present the `Wold's representation theorem` as the appropriate model to fit that series.

Theorem:

Let {yt} a zero-mean covariance-stationary process, we can write it as:

$$ y_t = B(L) \epsilon_t = \sum_{i=0}^{\infty} b_i \epsilon_{t-1} $$
$$ \epsilon_t \sim WN(0, \sigma^2) $$

where 

$$b_0 = 1$$
$$\sum_{i=0}^{\infty} b_i^2  < \infty$$


The `Wold representation` is an infinite distributed lag of white noise. The $\epsilon_t's$ are often called `innovations` for a good forecast, and they represent the part that is linearly unpredictable on the basis of the past of y.  

In the statistics literature, three models can approximate accurately and seamlessly to the `Wold's representation`: moving average models, autoregressive models and autoregressive moving average models. As we've dissected in this chapter, Autoregressive and Moving average models leverage past observations and white noises to explain the current observation, respectivelly. A ARMA model harnesses both at same time.   

A ARMA process combines two linear process: The Auto Regressive and the Moving Average, parameterized by two parameters: `p` e `q`. The mathematical notation goes as: 

$y_t - \phi_1y_{t-1} - ... - \phi_py_{t-p} = \epsilon_t + \theta_1\epsilon_{t-1} + ... + \theta_q\epsilon_{t-q} $

Therefore an ARMA leverages two polynomials $\phi$ and $\theta$ and their operation with past $x_{t-p}$ and $\epsilon_{t-q}$ values. On a vernacular tongue, the AR part relates the current value with the last results of the series, whereas the MA relates with respective random components of the oldest.

Two key concepts that raise here is `causal` and `independent`, which are deeply dig on the definitive guides.    


The properties of `point forecast` are similar to the AR and can easily be transmutated. 

------- 

### ARMA process

A ARMA(p,q) process is the foremost model for cycles. Conceptually, it generalizes and represents an AR process of infinite-order. To establish a polynomial representation, 

$$ \Phi (L)^p y_t =  \theta(L)^q \epsilon_t,$$ 

where the parameter `p` the sets maximum displacement for `AR`, whereas `q` for the `MA` model.